### Melon 100 Chart 
* 100곡의 노래의 제목, Song ID추출
* Song ID로 노래의 상세정보를 100 번 요청
* 100곡의 정보를 json에 저장
* json file을 읽어서 Pandas DataFrame에 저장해서 표데이터로 만들기
* 표데이터를 DB에 저장

In [41]:
import requests
from bs4 import BeautifulSoup
import re

req_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"
}
url = 'https://www.melon.com/chart/index.htm'

res = requests.get(url, headers=req_header)
#print(res.status_code)
soup = BeautifulSoup(res.text, 'html.parser')
# print(len(soup.select("a[href*='playSong']")))
# print(len(soup.select('div#tb_list'))) #1개
# print(len(soup.select('div#tb_list tr'))) #101
# print(len(soup.select('div#tb_list tr div.wrap_song_info'))) #200
# print(len(soup.select('div#tb_list tr div.wrap_song_info a'))) #427
# print(len(soup.select("div#tb_list tr div.wrap_song_info a[href*='playSong']"))) #100

#100곡의 song 정보를 저장할 list 선언
song_list = list()

atags = soup.select("div#tb_list tr div.wrap_song_info a[href*='playSong']")
for idx,atag in enumerate(atags,1):
    #1곡의 song 정보를 저장할 dict 선언
    song_dict = dict()
    title = atag.text
    song_dict['title'] = title
    href = atag['href']
    matched = re.search(r'(\d+)\);', href)
    song_id = matched.group(1)
    song_dict['id'] = song_id
    song_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
    #print(idx,title, song_url)
    song_dict['url'] = song_url
    #print(idx, song_dict)
    song_list.append(song_dict)

print(len(song_list))    
song_list[0:4]

100


[{'title': 'STAY',
  'id': '33658563',
  'url': 'https://www.melon.com/song/detail.htm?songId=33658563'},
 {'title': '신호등',
  'id': '33480898',
  'url': 'https://www.melon.com/song/detail.htm?songId=33480898'},
 {'title': 'My Universe',
  'id': '33998510',
  'url': 'https://www.melon.com/song/detail.htm?songId=33998510'},
 {'title': 'Permission to Dance',
  'id': '33666269',
  'url': 'https://www.melon.com/song/detail.htm?songId=33666269'}]

### 100곡의 Song의 상세정보 파싱하기

In [51]:
import requests
from bs4 import BeautifulSoup
import re

req_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"
}
#100곡의 Song 상세정보를 저장할 list 선언
song_detail_list = []
for idx,song in enumerate(song_list,1):
    #1곡의 Song 상세정보를 저장할 dict 선언
    song_detail_dict = {}
    res = requests.get(song['url'], headers=req_header)
    soup = BeautifulSoup(res.text, 'html.parser')
    print(f"----> {idx} = {song['title']}")
    #노래제목
    song_detail_dict['곡명'] = song['title']
    #가수
    song_detail_dict['가수'] = soup.select("a[href*='goArtistDetail'] span")[0].text
    #앨범
    song_detail_dict['앨범'] = soup.select('div.meta dd')[0].text
    #발매일
    song_detail_dict['발매일'] = soup.select('div.meta dd')[1].text
    #장르
    song_detail_dict['장르'] = soup.select('div.meta dd')[2].text
    #좋아요 건수
    song_id = song['id']
    like_url = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={song_id}'
    like_res = requests.get(like_url, headers=req_header)
    #print(song['title'], like_res.json()['contsLike'][0]['SUMMCNT'])
    song_detail_dict['좋아요'] = like_res.json()['contsLike'][0]['SUMMCNT']
    #가사
    lyric_tag = soup.select('div#d_video_summary')
    if lyric_tag:
        lyric = lyric_tag[0].text
        #print(lyric)
    else:
        lyric = ''
        
    #정규표현식 생성
    reg_exp = re.compile(r'[\n\r\t]')
    #정규표현식과 매핑되는 문자열을 empty string으로 대체해라
    lyric_clean = reg_exp.sub('',lyric.strip())
    song_detail_dict['가사'] = lyric_clean
    
    #dict을 list에 추가
    song_detail_list.append(song_detail_dict)

print(len(song_detail_list))
print(song_detail_list[0:3])


----> 1 = STAY
----> 2 = 신호등
----> 3 = My Universe
----> 4 = Permission to Dance
----> 5 = OHAYO MY NIGHT
----> 6 = Next Level
----> 7 = 바라만 본다
----> 8 = Butter
----> 9 = 낙하 (with 아이유)
----> 10 = Weekend
----> 11 = 좋아좋아
----> 12 = Dynamite
----> 13 = 이제 나만 믿어요
----> 14 = Queendom
----> 15 = 시간을 거슬러 (낮에 뜨는 달 X 케이윌)
----> 16 = Bad Habits
----> 17 = 다정히 내 이름을 부르면
----> 18 = Peaches (Feat. Daniel Caesar & Giveon)
----> 19 = DUMB DUMB
----> 20 = 헤픈 우연
----> 21 = 가을 타나 봐
----> 22 = 별빛 같은 나의 사랑아
----> 23 = 고백
----> 24 = 비와 당신
----> 25 = 그대라는 사치
----> 26 = HERO
----> 27 = 비가 오는 날엔 (2021)
----> 28 = 다시 사랑한다면 (김필 Ver.)
----> 29 = 라일락
----> 30 = 끝사랑
----> 31 = Dun Dun Dance
----> 32 = 롤린 (Rollin')
----> 33 = Sticker
----> 34 = Bk Love
----> 35 = 잊었니
----> 36 = Celebrity
----> 37 = Savage Love (Laxed - Siren Beat) (BTS Remix)
----> 38 = 가을 우체국 앞에서
----> 39 = ASAP
----> 40 = 색안경 (STEREOTYPE)
----> 41 = 작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey)
----> 42 = 하루만 더
----> 43 = 사이렌 Remix (Feat. UNEDUCATED

### Song 상세정보 리스트를 Json 저장

In [53]:
import json

with open('data/songs.json','w',encoding='utf8') as file:
    json.dump(song_detail_list, file)

In [56]:
import json

with open('data/songs.json','r',encoding='utf8') as file:
    contents = file.read()
    json_contents = json.loads(contents)

#json_contents    

### Json file을 읽어서 Pandas DataFrame으로 저장

In [60]:
import pandas as pd

song_df = pd.read_json('data/songs.json')
song_df.head()

,곡명,가수,앨범,발매일,장르,좋아요,가사
0,STAY,The Kid LAROI,Stay,2021.07.09,POP,155350,I do the same thing I told youthat I never wou...
1,신호등,이무진,신호등,2021.05.14,록/메탈,216683,이제야 목적지를 정했지만가려한 날 막아서네 난 갈 길이 먼데새빨간 얼굴로 화를 냈던...
2,My Universe,Coldplay,My Universe,2021.09.24,POP,81628,"You, you are my universe andI just want to put..."
3,Permission to Dance,방탄소년단,Butter / Permission to Dance,2021.07.09,댄스,177295,It’s the thought of being youngWhen your heart...
4,OHAYO MY NIGHT,디핵 (D-Hack),OHAYO MY NIGHT,2020.06.20,랩/힙합,135131,너를 사랑하고 있어너를 사랑하고 있어자기야 날 사랑해주면 안 될까말처럼 쉽지는 않은...


In [62]:
# song_detail_list을 읽어서 DataFrame 생성하기

# empty DataFrame을 생성
song_df2 = pd.DataFrame(columns=['곡명','가수','앨범','발매일','장르','좋아요','가사'])

for song_dict in song_detail_list:
    #dict를 Series객체로 wrapping
    series_obj = pd.Series(song_dict)
    #Series 객체를 DataFrame에 append 
    song_df2 = song_df2.append(series_obj, ignore_index=True)

song_df2.head()    

,곡명,가수,앨범,발매일,장르,좋아요,가사
0,STAY,The Kid LAROI,Stay,2021.07.09,POP,155350,I do the same thing I told youthat I never wou...
1,신호등,이무진,신호등,2021.05.14,록/메탈,216683,이제야 목적지를 정했지만가려한 날 막아서네 난 갈 길이 먼데새빨간 얼굴로 화를 냈던...
2,My Universe,Coldplay,My Universe,2021.09.24,POP,81628,"You, you are my universe andI just want to put..."
3,Permission to Dance,방탄소년단,Butter / Permission to Dance,2021.07.09,댄스,177295,It’s the thought of being youngWhen your heart...
4,OHAYO MY NIGHT,디핵 (D-Hack),OHAYO MY NIGHT,2020.06.20,랩/힙합,135131,너를 사랑하고 있어너를 사랑하고 있어자기야 날 사랑해주면 안 될까말처럼 쉽지는 않은...
